# Plotly/Dash Dashboard
## Analysing SpaceX data and deploying an interactive dashboard
**Notebook for the Dash/PLotly lab**  
*Created locally by me to complete the session, given the the actual online service for the lab was down*

**Code block required to get wget, in case it wasn't installed**

    !pip install wget
    
    import wget
    
    #Getting the dataset we will be using
    
    wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

## Now, the actual notebook

In [1]:
import dash
import plotly.express as px
import numpy as np
import pandas as pd

In [2]:
from dash import Dash, dcc, html, no_update
from dash.dependencies import Input, Output, State

In [3]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

### 1st part - Launch site v succes rate
**Creating the first part of the app**  
Adding the pie chart to represent the succes rate of launches per launch site

In [4]:
app = Dash('SpaceX Dashboard')


# Let's create the customized options, to include the all sites keayword
dropdown_options = [{'label':'All Sites', 'value':'ALL'},]
dropdown_options.extend([{'label':lsite,'value':lsite} for lsite in df['Launch Site'].unique()])

app.layout = html.Div([
    dcc.Dropdown(
        options = dropdown_options,
        value = 'ALL',
        multi = False,
        id = 'launch_selected'
    ),
    dcc.Graph(id='pie-chart'),
])

@app.callback(
    Output('pie-chart', 'figure'),
    Input('launch_selected', 'value'))
def update_figure(selected_launchSite):
    """
    Callback method that renders a pie chart.
    If the option All Sites is selected (keyword ALL) -> the success rate for each site is rendered
    If an specific site  is selected (keyword Name of site) -> the % success vs % failure is rendered
    """
    if selected_launchSite is None:
        #Safeguard in case of breaking the dropdown somehow
        return no_update
    
    if selected_launchSite == 'ALL':
        #No filtering needed here
        return px.pie(
            data_frame = df,
            names='Launch Site',
            values='class',
            title= f'Total Success Launches by site'
        )
    
    #Not and else required, given that the return is hit previously
    filtered_df = df[df['Launch Site'] == selected_launchSite]
    return px.pie(
        data_frame = filtered_df,
        names='class',
        title= f'Total Success Launches for site {selected_launchSite}'
    )



#app.run_server(debug=True)

In [9]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'SpaceX-Dashboard'
 * Debug mode: on


SystemExit: 1

d:\Anaconda3\envs\dash-app-tutorial\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



### 2nd part - Payload mass v Success class for launches
**Creating the second iteration of the app**  
Adding the scatter plot for the payload mass v success launches

In [6]:
min_payload = df['Payload Mass (kg)'].min()
max_payload = df['Payload Mass (kg)'].max()

In [8]:
int(min_payload)

0

In [11]:
app = Dash('SpaceX Dashboard')



#Adding some customization to the marks - Colour scale ....
#color_list = ["#7FFF00","#FFD700","#FF8C00","#FF0000","#8B2252"]
#marks_styled_1 = {int(el) : {"label":f"{int(el)} kg", "style":{"color":color_list[i]}} for i,el in enumerate(np.linspace(0,10000,5))}
marks_styled = {str(int(el)):{"label": f"{int(el)}","style":{"color":"#228B22"}} for el in np.linspace(0,10000,5)}
marks_styled[str(int(min_payload))] =  {"label": f"Minimum : {int(min_payload)}","style":{"color":"#228B22"}}
marks_styled[str(int(max_payload))] =  {"label": f"Maximum : {int(max_payload)}","style":{"color":"#DC143C"}}

app.layout = html.Div([
    dcc.RangeSlider(
        id='payload-slider',
        min=0, 
        max=10000, 
        step=1000,
        marks=marks_styled,
        #marks = {int(el):f"{int(el)}" for el in np.linspace(0,10000,5)},
        value=[min_payload, max_payload],
        tooltip={"placement": "bottom", "always_visible": True},
        
    ),
    dcc.Graph(id="success-payload-scatter-chart"),
])

@app.callback(
    Output("success-payload-scatter-chart", 'figure'),
    Input('payload-slider', 'value'))
def update_figure(selected_payloadRange):
    """
    Callback method that renders a scatter chart.
    The graph shows the successfull launches filtered by the
    range of payload mass [in kg] selected
    """
    if selected_payloadRange is None:
        #Safeguard in case of breaking the slider somehow
        return no_update    
    
    #Not and else required, given that the return is hit previously
    filtered_df = df[df['Payload Mass (kg)'].isin(range(*selected_payloadRange))]
    return px.scatter(filtered_df,
        x = 'Payload Mass (kg)',
        y = 'class',
        color = 'Booster Version Category',
        title= 'Correlation between Payload and Success for all Sites'
        )



#app.run_server(debug=True)

In [12]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'SpaceX Dashboard'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit


## Complete app

**Now, let's bring all together**

The only thing left here is to bring all the functinoalities together in a single dashboard.


**Imports**

In [4]:
#import dash
import plotly.express as px
import numpy as np
import pandas as pd
from dash import Dash, dcc, html, no_update
from dash.dependencies import Input, Output

from jupyter_dash import JupyterDash

**Loading dataset directly from the URL**

In [5]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

**App structure**

In [6]:
app = Dash('SpaceX-Dashboard')
#app = JupyterDash('SpaceX-Dashboard')

**Layout**

In [7]:
# Let's create the customized options, to include the all sites keayword
dropdown_options = [{'label':'All Sites', 'value':'ALL'},]
dropdown_options.extend([{'label':lsite,'value':lsite} for lsite in df['Launch Site'].unique()])

#Now the customization for the scatter plot and the range slider
min_payload = df['Payload Mass (kg)'].min()
max_payload = df['Payload Mass (kg)'].max()

#Adding some customization to the marks - Colour scale ....
marks_styled = {int(el):{"label": f"{int(el)}","style":{"color":"#000000"}} for el in np.linspace(0,10000,5)}
marks_styled[int(min_payload)] =  {"label": f"Minimum : {int(min_payload)}","style":{"color":"#228B22"}}
marks_styled[int(max_payload)] =  {"label": f"Maximum : {int(max_payload)}","style":{"color":"#DC143C"}}

#The layout
app.layout = html.Div(
    [
        html.H1("SpaceX Launch Records Dashboard"),
        html.Hr(),
        dcc.Dropdown(
            options = dropdown_options,
            value = 'ALL',
            multi = False,
            id = 'launch-selected'
        ),
        dcc.Graph(id='pie-chart'),
        html.Hr(),
        dcc.Graph(id="success-payload-scatter-chart"),
        dcc.RangeSlider(
            id='payload-slider',
            min=0, 
            max=10000, 
            step=1000,
            marks=marks_styled,
            value=[min_payload, max_payload],
            tooltip={"placement": "top", "always_visible": True},

        ),
    ]
)

**Callbacks**

In [8]:
#Callbacks
@app.callback(
    Output('pie-chart', 'figure'),
    Input('launch-selected', 'value'))
def update_pie_figure(selected_launchSite):
    """
    Callback method that renders a pie chart.
    If the option All Sites is selected (keyword ALL) -> the success rate for each site is rendered
    If an specific site  is selected (keyword Name of site) -> the % success vs % failure is rendered
    """
    if selected_launchSite is None:
        #Safeguard in case of breaking the dropdown somehow
        return no_update
    
    if selected_launchSite == 'ALL':
        #No filtering needed here
        return px.pie(
            data_frame = df,
            names='Launch Site',
            values='class',
            title= f'Total Success Launches by site'
        )
    
    #Not and else required, given that the return is hit previously
    filtered_df = df[df['Launch Site'] == selected_launchSite]
    return px.pie(
        data_frame = filtered_df,
        names='class',
        title= f'Total Success Launches for site {selected_launchSite}'
    )

@app.callback(
    Output("success-payload-scatter-chart", 'figure'),
    Input('payload-slider', 'value'))
def update_scatter_figure(selected_payloadRange):
    """
    Callback method that renders a scatter chart.
    The graph shows the successfull launches filtered by the
    range of payload mass [in kg] selected
    """
    if selected_payloadRange is None:
        #Safeguard in case of breaking the slider somehow
        return no_update    
    
    #Not and else required, given that the return is hit previously
    filtered_df = df[df['Payload Mass (kg)'].isin(range(*selected_payloadRange))]
    return px.scatter(filtered_df,
        x = 'Payload Mass (kg)',
        y = 'class',
        color = 'Booster Version Category',
        title= 'Correlation between Payload and Success for all Sites'
        )

**Running the server**

In [9]:
#app.run_server(mode = 'external') # Case of using JupyterDash instance for the app
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'SpaceX-Dashboard'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [11/Mar/2023 13:13:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2023 13:13:51] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_1m1659522771.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2023 13:13:51] "GET /_dash-component-suites/dash/deps/react@16.v2_6_1m1659522771.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2023 13:13:51] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_1m1659522771.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2023 13:13:51] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_1m1659522771.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2023 13:13:51] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_1m1659522771.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2023 13:13:51] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_6_1m1659522771.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2023 13:13:51] "GET /_dash-component-suites/dash

### Final thoughts

There are better ways of running this exercise.

For once, the JupyterDash interface would be one of them, since it allows to run the servers inline with the notebok, or in a separated window still embedded in the Jupyter Lab ecosystem.

Also, scripting in python is a nicer way to create plotly-dash apps. As a matter of fact, in the same repo (GitHub) that hosts this notebook, I have an alternative version of the dahsboard that can be launched from a terminal

To run the jupyter dash version, we only have to intall the following (through conda in my case)

```bash
$ conda install -c conda-forge jupyter-dash
```

And substitute the line

```python
from dash import Dash
title = 'SpaceX Landing app'
app = Dash(title)
```

by 

```python
from jupyter_dash import JupyterDash

title = 'SpaceX Landing app'
app = JupyterDash(title)
```

Then, the server can be run as follows

```python
# For the inline mode
app.run_server(mode = 'inline')
```

or for the mode embeded in jupyterlab

```python
# For the inline mode
app.run_server(mode='jupyterlab')
```

Where both cases will implement hot-reload if given the `debug = True` keyword in `run_server(*args,**kwargs)`, that uppon re-running the block with the `run_server()` statement will refresh the view of the app.

In the thrid and final case - external mode

```python
# For the inline mode
app.run_server(mode='external', debug = True)
```

the hot reloaded is implemented in a way such as the own app will wacth for changes in the code an refresh when detected.


